In [ ]:
from serpapi import GoogleSearch
import json
title1=[]
product_id1=[]
category1=[]
description1=[]
link1=[]
DEVICE_CATEGORIES = ['windows', 'tv', 'tablet', 'phone', 'chromebook']

for i in DEVICE_CATEGORIES:
    
    params = {
        'api_key': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',#put your serp api keys here
        'engine': 'google_play',
        'store': 'games',  # Explicitly set the category to games
        'store_device': i,  # Specify device type
         }

    search = GoogleSearch(params)   # where data extraction happens on the SerpApi backend
    result_dict = search.get_dict() # JSON -> Python dict

    #print(result_dict)
    google_play_games = result_dict['organic_results']
    
    
    for j in range(len(google_play_games)):
        for k in range(len(google_play_games[j]["items"])):
            title1.append(google_play_games[j]["items"][k]["title"])
            product_id1.append(google_play_games[j]["items"][k]["product_id"])
            category1.append(google_play_games[j]["items"][k]["category"])
            description1.append(google_play_games[j]["items"][k]["description"])
            link1.append(google_play_games[j]["items"][k]["link"])
        
    
    
    print(f'{i}done*************************************************************')

In [ ]:
import pandas as pd 
dataset= pd.DataFrame({
    'title': title1,
    'product_id':product_id1,
    'category': category1,
    'description':description1,
    'link': link1
})

print(dataset)

In [ ]:
dataset.to_csv(r'C:\Users\Dell\3D Objects\NLP\nowgg_dataset3.csv', index=False)

In [ ]:
import pandas as pd
dataset1=pd.read_csv("nowgg_dataset3.csv")
dataset1

In [ ]:
dataset2=dataset1.drop_duplicates()

In [ ]:
dataset2

In [ ]:
dataset2['category'] = dataset2['category'].str.replace(' ', '')#remove spaces between words to capture semantic meaning

In [ ]:
dataset2["category"]=dataset2["category"].apply(lambda x:x.lower())

In [ ]:
#Here the cleaning of description columns take place
import ast
import string
import re

def cleaning(text):
    
    
    translator = str.maketrans('', '', string.punctuation)
    text3=text.translate(translator)#punctuations
    text1=text3.lower()#lower
    text2 = re.sub(r'\d+', '', text1)
    text4=" ".join(text2.split())#join
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text5= emoji_pattern.sub(r'', text4)
    text5=text5.replace('•','')
    
    
    dem = demoji.findall(text5)

    for item in dem.keys():

        text5 = text5.replace(item,'')
        
    text5=re.sub("[^a-z]",' ',text5)
    

    return text5


dataset2["description"]=dataset2["description"].apply(cleaning)

In [ ]:
dataset2

In [ ]:
dataset2["description"]=dataset2["description"].apply(lambda x:x.split()[:100])
dataset2["category"]=dataset2["category"].apply(lambda x:x.split())
dataset2["final_description"]=dataset2["category"]+dataset2["description"]
dataset2=dataset2.drop(["category","description"],axis=1)

In [ ]:
dataset2

In [ ]:
dataset2["final_description"]=dataset2["final_description"].apply(lambda x:" ".join(x))

In [ ]:
dataset2

In [ ]:
dataset3=dataset2.dropna()

In [ ]:
dataset3 = dataset3.reset_index(drop=True)
dataset3.isnull().sum()

In [ ]:
dataset3["embeddings"]=None

In [ ]:
import cohere           
import pandas as pd

# Initialize Cohere client
cohere_api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Replace with your Cohere API key
co = cohere.Client(cohere_api_key)


# Function to get embeddings using Cohere
def get_cohere_embedding(text):
    response = co.embed(texts=[text])
    return response.embeddings[0][:250]

embeddings = []

for index,row in dataset3.iterrows():
    if row["final_description"]!="NaN":
        embedding = get_cohere_embedding(row['final_description'])
        embeddings.append(embedding)
        print(f"{index}done")


In [ ]:
for i in range(len(embeddings)):
    dataset3.at[i,"embeddings"]=embeddings[i]

In [ ]:
dataset3

In [ ]:
dataset3.to_csv(r'C:\Users\Dell\3D Objects\NLP\nowgg_embeddings.csv', index=False)

In [ ]:
import faiss
import numpy as np

# Convert embeddings to a numpy array
embedding_matrix = np.array(dataset3['embeddings'].tolist()).astype('float32')

In [ ]:
# Create a FAISS index
dimension = embedding_matrix.shape[1]  # Dimensionality of the embeddings
index = faiss.IndexFlatL2(dimension)  # L2 distance index

In [ ]:
index.add(embedding_matrix)

# Check the number of elements in the index
print(f"Number of embeddings indexed: {index.ntotal}")

In [ ]:
faiss.write_index(index, "faiss_index.bin")